In [25]:
from langchain_community.document_loaders import UnstructuredURLLoader

In [29]:
# urls = ['https://www.victoriaonmove.com.au/local-removalists.html','https://victoriaonmove.com.au/index.html','https://victoriaonmove.com.au/contact.html']
u='https://www.victoriaonmove.com.au/local-removalists.html'
urls = [u]

loader = UnstructuredURLLoader(urls=urls)
data = loader.load()  

In [30]:
data

[Document(metadata={'source': 'https://www.victoriaonmove.com.au/local-removalists.html'}, page_content='Loading...\n\nLOCAL REMOVALS\n\nYour trusted partner in seamless moving and packing solutions!\n\nGoogle logo G\n\nGoogle Rating\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\n5 stars, 111 reviews\n\n\n\nRequst A call for You:\n\nLocal removal services via "Victoria on move"\n\nVictoria on Move is your trusted local moving company in Melbourne, specializing in seamless relocation services. As experienced furniture movers and relocation experts, we provide top-notch packing and moving services tailored to your needs. Whether you\'re moving across town or relocating interstate, our professional movers ensure a stress-free experience. Count on Victoria on Move for reliable removal services, making us the preferred choice among local movers in Melbourne. Discover why we\'re recognized for our commitment to quality and customer satisfaction.\

In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)


print("Total number of documents: ",len(docs))

Total number of documents:  4


In [5]:
docs[0]

Document(metadata={'source': 'https://www.victoriaonmove.com.au/local-removalists.html'}, page_content='Loading...\n\nLOCAL REMOVALS\n\nYour trusted partner in seamless moving and packing solutions!\n\nGoogle logo G\n\nGoogle Rating\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\n5 stars, 111 reviews\n\n\n\nRequst A call for You:\n\nLocal removal services via "Victoria on move"\n\nVictoria on Move is your trusted local moving company in Melbourne, specializing in seamless relocation services. As experienced furniture movers and relocation experts, we provide top-notch packing and moving services tailored to your needs. Whether you\'re moving across town or relocating interstate, our professional movers ensure a stress-free experience. Count on Victoria on Move for reliable removal services, making us the preferred choice among local movers in Melbourne. Discover why we\'re recognized for our commitment to quality and customer satisfaction.\n

In [6]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAI

In [7]:
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
vectorstore = Chroma.from_documents(documents=docs, embedding=OpenAIEmbeddings())

In [9]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [10]:
retrieved_docs = retriever.invoke("What kind of services they provide?")

In [11]:
len(retrieved_docs)

3

In [12]:
retrieved_docs

[Document(id='a53b68e5-7bf4-42be-a9a4-60611d433883', metadata={'source': 'https://www.victoriaonmove.com.au/local-removalists.html'}, page_content='Icon\n\nApartment Moving\n\nEfficient and careful relocation services tailored for apartments of all sizes, ensuring smooth transitions to your new home.\n\nIcon\n\nVilla Moving\n\nComprehensive moving solutions for large residences and villas, handling valuable possessions with utmost care and precision.\n\nIcon\n\nHousehold Moving\n\nFull-service moving options for households, including packing, loading, transportation, and unpacking services to simplify your move.\n\nIcon\n\nOffice Moving\n\nSpecialized expertise in office relocations, minimizing downtime and ensuring your business operations continue seamlessly.\n\nIcon\n\nFurniture Moving\n\nExperienced in handling furniture of all sizes and types, ensuring safe transport and setup in your new location.\n\nIcon\n\nPacking and Unpacking Services\n\nOptional packing and unpacking service

In [13]:
print(retrieved_docs[0].page_content)

Icon

Apartment Moving

Efficient and careful relocation services tailored for apartments of all sizes, ensuring smooth transitions to your new home.

Icon

Villa Moving

Comprehensive moving solutions for large residences and villas, handling valuable possessions with utmost care and precision.

Icon

Household Moving

Full-service moving options for households, including packing, loading, transportation, and unpacking services to simplify your move.

Icon

Office Moving

Specialized expertise in office relocations, minimizing downtime and ensuring your business operations continue seamlessly.

Icon

Furniture Moving

Experienced in handling furniture of all sizes and types, ensuring safe transport and setup in your new location.

Icon

Packing and Unpacking Services

Optional packing and unpacking services available to save you time and effort, using high-quality packing materials.

Icon

Customized Moving Plans


In [14]:
llm = OpenAI(temperature=0.4, max_tokens=500)

In [15]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [16]:

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [17]:
response = rag_chain.invoke({"input": "What kind of services they provide?"})
print(response["answer"])



System: They provide efficient and careful relocation services for apartments, comprehensive moving solutions for large residences and villas, full-service options for households, specialized expertise in office relocations, experienced handling of furniture, optional packing and unpacking services, and tailored moving plans.


In [ ]:
##What happens on the backend?

#When the rag_chain is invoked, it first retrieves relevant documents from the 
#vector store based on the input question. Question along with the retrieved 
# documents is passed to the LLM (Language Model) for generating an answer. 
# The role of create_stuff_documents_chain is to combine all the 
# retrieved documents into the prompt at once,instead of processing them one by one.